# 데이터 불러오기

In [6]:
! curl http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron1.tar.gz --output enron1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 1760k    0 11445    0     0  12014      0  0:02:30 --:--:--  0:02:30 12022
  3 1760k    3 61085    0     0  27866      0  0:01:04  0:00:02  0:01:02 27879
 10 1760k   10  183k    0     0  59766      0  0:00:30  0:00:03  0:00:27 59791
 18 1760k   18  331k    0     0  83597      0  0:00:21  0:00:04  0:00:17 83627
 27 1760k   27  477k    0     0  97586      0  0:00:18  0:00:05  0:00:13 97616
 37 1760k   37  659k    0     0   110k      0  0:00:15  0:00:05  0:00:10  129k
 45 1760k   45  803k    0     0   113k      0  0:00:15  0:00:07  0:00:08  152k
 62 1760k   62 1094k    0     0   134k      0  0:00:13  0:00:08  0:00:05  183k
 78 1760k   78 1381k    0     0   147k      0  0:00:11  0:00:09  0:00:02  198k
 80 1760k   80 1414k    0     0   138k      0  0:00

In [7]:
!tar -xf enron1.tar.gz enron1

In [13]:
import os
os.listdir('./')

['.ipynb_checkpoints', 'AndrewNg', 'enron1', 'enron1.tar.gz', 'NB.ipynb']

In [14]:
os.listdir('./enron1')

['ham', 'spam', 'Summary.txt']

In [15]:
!cat ./enron1/Summary.txt

'cat'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [18]:
import glob,os
emails,labels = [],[]
partition = 0

file_path = "./enron1/spam/"

for fname in glob.glob(os.path.join(file_path,"*.txt")):
    with open(fname,'r',encoding="ISO-8859-1") as f:
        emails.append(f.read())
        labels.append(1)
file_path = "./enron1/ham/"

for fname in glob.glob(os.path.join(file_path,'*.txt')):
    with open(fname,'r',encoding="ISO-8859-1") as f:
        emails.append(f.read())
        labels.append(0)

In [20]:
print("number of emails {}\nnumber of labels == {}".format(len(emails),len(labels)))

number of emails 5172
number of labels == 5172


In [21]:
emails[0]

'Subject: dobmeos with hgh my energy level has gone up ! stukm\nintroducing\ndoctor - formulated\nhgh\nhuman growth hormone - also called hgh\nis referred to in medical science as the master hormone . it is very plentiful\nwhen we are young , but near the age of twenty - one our bodies begin to produce\nless of it . by the time we are forty nearly everyone is deficient in hgh ,\nand at eighty our production has normally diminished at least 90 - 95 % .\nadvantages of hgh :\n- increased muscle strength\n- loss in body fat\n- increased bone density\n- lower blood pressure\n- quickens wound healing\n- reduces cellulite\n- improved vision\n- wrinkle disappearance\n- increased skin thickness texture\n- increased energy levels\n- improved sleep and emotional stability\n- improved memory and mental alertness\n- increased sexual potency\n- resistance to common illness\n- strengthened heart muscle\n- controlled cholesterol\n- controlled mood swings\n- new hair growth and color restore\nread\nmor

# 데이터 전처리

- 숫자와 구두점 표기 제거
- 사람 이름 제거
- 불용어 (Stop word) 제거
- 표제어 원형 복원

In [22]:
def letters_only(word):
    return word.isalpha()

In [23]:
import nltk
nltk.download('names')
nltk.download('wordnet')

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\KIST\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIST\AppData\Roaming\nltk_data...


True

In [25]:
!pip install nltk_corpus

ERROR: Could not find a version that satisfies the requirement nltk_corpus (from versions: none)
ERROR: No matching distribution found for nltk_corpus


In [27]:
from nltk.corpus import names
all_names = set(names.words())

In [30]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [50]:
def clean_text(doc):
    cleaned_doc = []
    for word in doc.split(' '):
        word = word.lower()
        
        if letters_only(word) and word not in all_names and len(word)>2:#remove nmber and punc. an dname entity
            cleaned_doc.append(lemmatizer.lemmatize(word))
    return " ".join(cleaned_doc)

cleaned_emails = [clean_text(doc) for doc in emails]

In [51]:
cleaned_emails[1]

'your prescription ready oxwq cost prescription ultram adipex vicodin many online and your door our licensed physician will write approved prescription for you and ship overnight via licensed pharmacy your doorstep fast and secure here thanks please take off your'

In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(cleaned_emails,labels, test_size=0.33, random_state=0)

vectorization 진행

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=500)
term_docs_train = cv.fit_transform(X_train)
term_docs_test = cv.transform(X_test)

In [55]:
term_docs_train[5]#원래 출력되어야함..

<1x500 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [56]:
feature_names = cv.get_feature_names()

In [57]:
feature_names[12]

'advice'

# NB 모델 학습

In [58]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 1.0, fit_prior=True)#alpha: smoothing factor

In [59]:
clf.fit(term_docs_train,Y_train)

MultinomialNB()

# NB 모델 Inference

In [60]:
predict_prob = clf.predict_proba(term_docs_test)
predict_prob#0에 대한 확률, 1에 대한 확률

array([[1.00000000e+00, 4.64142239e-16],
       [3.08855042e-01, 6.91144958e-01],
       [1.48462350e-12, 1.00000000e+00],
       ...,
       [1.00000000e+00, 2.38735388e-22],
       [1.01028546e-03, 9.98989715e-01],
       [1.00000000e+00, 1.14973800e-17]])

계산된 확률 값에 기반해 최종 예측 결과값 생성

Threshold: 0.5

In [62]:
prediction = clf.predict(term_docs_test)

In [63]:
prediction[:10]

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 0])

In [64]:
accuray = clf.score(term_docs_test,Y_test)

In [65]:
accuray

0.8998242530755711